In [15]:
from google.colab import drive

drive.mount('/content/drive/')
%cd drive/MyDrive/CTR_prediction

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/CTR_prediction'
/content/drive/MyDrive/CTR_prediction


In [ ]:
#!pip install deepctr[gpu]
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple deepctr[cpu] tensorflow==2.5.0

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

DeepCTR version 0.9.0 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


In [ ]:
columns = ['label', *(f'I{i}' for i in range(1, 14)), *(f'C{i}' for i in range(1, 27))]
data = pd.read_csv('./data/train_1m.txt', sep='\t', names=columns)

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I'+str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

In [ ]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                           for i,feat in enumerate(sparse_features)]
dense_feature_columns = [DenseFeat(feat, 1)
                      for feat in dense_features]

In [ ]:
#varlen_feature_columns = [VarLenSparseFeat('genres', maxlen=max_len,vocabulary_size= len(key2index) + 1,embedding_dim=4, combiner='mean',weight_name=None)]

dnn_feature_columns = sparse_feature_columns + dense_feature_columns
linear_feature_columns = sparse_feature_columns + dense_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
train, test = train_test_split(data, test_size=0.2)

train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}


model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary', dnn_use_bn = True, l2_reg_dnn = 0.005, dnn_dropout = 0.3)

model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy', 'accuracy'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=50, verbose=2, validation_split=0.2, )

pred_ans = model.predict(test_model_input, batch_size=256)

Epoch 1/50
2500/2500 - 211s - loss: 0.5683 - binary_crossentropy: 0.4929 - accuracy: 0.7681 - val_loss: 0.4990 - val_binary_crossentropy: 0.4777 - val_accuracy: 0.7742
Epoch 2/50
2500/2500 - 145s - loss: 0.4735 - binary_crossentropy: 0.4371 - accuracy: 0.8024 - val_loss: 0.5223 - val_binary_crossentropy: 0.4806 - val_accuracy: 0.7743
Epoch 3/50
2500/2500 - 145s - loss: 0.4303 - binary_crossentropy: 0.3862 - accuracy: 0.8296 - val_loss: 0.5474 - val_binary_crossentropy: 0.5023 - val_accuracy: 0.7573
Epoch 4/50
2500/2500 - 145s - loss: 0.3917 - binary_crossentropy: 0.3481 - accuracy: 0.8446 - val_loss: 0.5812 - val_binary_crossentropy: 0.5369 - val_accuracy: 0.7565
Epoch 5/50
2500/2500 - 144s - loss: 0.3890 - binary_crossentropy: 0.3453 - accuracy: 0.8455 - val_loss: 0.5788 - val_binary_crossentropy: 0.5334 - val_accuracy: 0.7634
Epoch 6/50
2500/2500 - 145s - loss: 0.3917 - binary_crossentropy: 0.3469 - accuracy: 0.8450 - val_loss: 0.5757 - val_binary_crossentropy: 0.5292 - val_accuracy:

In [ ]:
loss = history.history['loss']

In [ ]:
#plt.plot(range(1,101), loss)
plt.plot(range(1,101), history.history['val_binary_crossentropy'])

In [ ]:
history.history.keys()

In [ ]:
pred_ans

In [ ]:
model.summary()

In [ ]:
model.save_weights('./')